In [1]:
import os
import sys
import pandas as pd

package_path = '/Users/flaminia/Documents/work/ProMCDA'

if package_path not in sys.path:
    sys.path.append(package_path)

try:
    from mcda.models.ProMCDA import ProMCDA
    print("Import successful!")
except ModuleNotFoundError as e:
    print(f"ModuleNotFoundError: {e}")

from mcda.configuration.enums import NormalizationFunctions, AggregationFunctions, OutputColumnNames4Sensitivity, NormalizationNames4Sensitivity, PDFType

Import successful!


In [2]:
def setUpNoRobustnessIndicators():
    
    # Mock input data for testing
    input_matrix_without_uncertainty = pd.DataFrame({
        'Alternatives': ['A', 'B', 'C'],
        'Criteria 1': [0.5, 0.2, 0.8],
        'Criteria 2': [0.3, 0.6, 0.1]
    }, index=['A', 'B', 'C'])
   
    input_matrix_without_uncertainty.set_index('Alternatives', inplace=True)

    polarity = ('+', '-')

    robustness_weights = False
    robustness_indicators = False

    marginal_distributions = [PDFType.NORMAL, PDFType.NORMAL]

    num_runs = 5
    num_cores = 1

    output_path = 'mock_output/'

    # Return the setup parameters as a dictionary
    return {
        'input_matrix': input_matrix_without_uncertainty, # Decide what type of input matrix
        'polarity': polarity,
        'robustness_weights': robustness_weights,
        'robustness_indicators': robustness_indicators,
        'marginal_distributions': marginal_distributions,
        'num_runs': num_runs,
        'num_cores': num_cores,
        'output_path': output_path
    }

# Run the setup and store parameters in a variable
setup_no_robustness_indicators = setUpNoRobustnessIndicators()

# Check the setup parameters
setup_no_robustness_indicators

{'input_matrix':               Criteria 1  Criteria 2
 Alternatives                        
 A                    0.5         0.3
 B                    0.2         0.6
 C                    0.8         0.1,
 'polarity': ('+', '-'),
 'robustness_weights': False,
 'robustness_indicators': False,
 'marginal_distributions': [<PDFType.NORMAL: 'normal'>,
  <PDFType.NORMAL: 'normal'>],
 'num_runs': 5,
 'num_cores': 1,
 'output_path': 'mock_output/'}

In [3]:
def setUpRobustnessIndicators():
    
    # Mock input data for testing
    input_matrix_with_uncertainty = pd.DataFrame({
    'Alternatives': ['A', 'B', 'C'],
    'Criterion1_mean': [0.5, 0.2, 0.8],
    'Criterion1_std': [0.1, 0.02, 0.07],
    'Criterion2_mean': [0.3, 0.6, 0.1],
    'Criterion2_std': [0.03, 0.06, 0.01]
        })
   
    input_matrix_with_uncertainty.set_index('Alternatives', inplace=True)

    polarity = ('+', '-')

    robustness_weights = False
    robustness_indicators = True

    marginal_distributions = [PDFType.NORMAL, PDFType.NORMAL]

    num_runs = 5
    num_cores = 1

    output_path = 'mock_output/'

    # Return the setup parameters as a dictionary
    return {
        'input_matrix': input_matrix_with_uncertainty, # Decide what type of input matrix
        'polarity': polarity,
        'robustness_weights': robustness_weights,
        'robustness_indicators': robustness_indicators,
        'marginal_distributions': marginal_distributions,
        'num_runs': num_runs,
        'num_cores': num_cores,
        'output_path': output_path
    }

# Run the setup and store parameters in a variable
setup_robustness_indicators = setUpRobustnessIndicators()

# Check the setup parameters
setup_robustness_indicators

{'input_matrix':               Criterion1_mean  Criterion1_std  Criterion2_mean  Criterion2_std
 Alternatives                                                                  
 A                         0.5            0.10              0.3            0.03
 B                         0.2            0.02              0.6            0.06
 C                         0.8            0.07              0.1            0.01,
 'polarity': ('+', '-'),
 'robustness_weights': False,
 'robustness_indicators': True,
 'marginal_distributions': [<PDFType.NORMAL: 'normal'>,
  <PDFType.NORMAL: 'normal'>],
 'num_runs': 5,
 'num_cores': 1,
 'output_path': 'mock_output/'}

## TEST NO ROBUSTNESS ON INDICATORS

In [4]:
promcda = ProMCDA(
    input_matrix=setup_no_robustness_indicators['input_matrix'],
    polarity=setup_no_robustness_indicators['polarity'],
    robustness_weights=setup_no_robustness_indicators['robustness_weights'],
    robustness_indicators=setup_no_robustness_indicators['robustness_indicators'],
    marginal_distributions=setup_no_robustness_indicators['marginal_distributions'],
    num_runs=setup_no_robustness_indicators['num_runs'],
    num_cores=setup_no_robustness_indicators['num_cores'],
    #output_path=setup_parameters['output_path']
)

INFO: 2024-12-19 13:56:56,169 - ProMCDA - Alternatives are ['A', 'B', 'C']


### Test normalize with sensitivity¶

In [5]:
promcda.normalize()

,Criteria 1_minmax_01,Criteria 2_minmax_01,Criteria 1_minmax_without_zero,Criteria 2_minmax_without_zero,Criteria 1_target_01,Criteria 2_target_01,Criteria 1_target_without_zero,Criteria 2_target_without_zero,Criteria 1_standardized_any,Criteria 2_standardized_any,Criteria 1_standardized_without_zero,Criteria 2_standardized_without_zero,Criteria 1_rank,Criteria 2_rank
0,0.5,0.6,0.55,0.64,0.625,0.5,0.6625,0.55,0.0,0.132453,1.1,1.292079,2.0,2.0
1,0.0,0.0,0.1,0.1,0.25,0.0,0.325,0.1,-1.0,-1.059626,0.1,0.1,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,0.833333,1.0,0.85,1.0,0.927173,2.1,2.086799,3.0,3.0


### Test normalize with specific method

In [6]:
promcda.normalize(NormalizationFunctions.MINMAX)

,Criteria 1_minmax_01,Criteria 2_minmax_01,Criteria 1_minmax_without_zero,Criteria 2_minmax_without_zero
0,0.5,0.6,0.55,0.64
1,0.0,0.0,0.1,0.1
2,1.0,1.0,1.0,1.0


In [7]:
promcda.normalize(NormalizationFunctions.RANK)

,Criteria 1_rank,Criteria 2_rank
0,2.0,2.0
1,1.0,1.0
2,3.0,3.0


### Test aggregate with full sensitivity

In [8]:
promcda.normalize()
promcda.aggregate()

INFO: 2024-12-19 13:58:42,150 - ProMCDA - Number of alternatives: 3
INFO: 2024-12-19 13:58:42,153 - ProMCDA - Number of indicators: 2
INFO: 2024-12-19 13:58:42,154 - ProMCDA - Polarities: ('+', '-')
INFO: 2024-12-19 13:58:42,154 - ProMCDA - Weights: [0.5, 0.5]
INFO: 2024-12-19 13:58:42,154 - ProMCDA - Normalized weights: [0.5, 0.5]


,minmax_weighted_sum,target_weighted_sum,standardized_weighted_sum,rank_weighted_sum,minmax_geometric,target_geometric,standardized_geometric,rank_geometric,minmax_harmonic,target_harmonic,standardized_harmonic,rank_harmonic,standardized_minimum
0,0.55,0.5625,0.066227,2.0,0.593296,0.603635,1.192177,2.0,0.591597,0.601031,1.188328,2.0,0.0
1,0.0,0.125,-1.029813,1.0,0.100000,0.180278,0.100000,1.0,0.1,0.152941,0.1,1.0,-1.059626
2,1.0,0.916667,0.963586,3.0,1.000000,0.921954,2.093389,3.0,1.0,0.918919,2.093378,3.0,0.927173


### Test aggregate with sensitivity on aggregation

In [9]:
promcda.normalize(NormalizationFunctions.MINMAX)
promcda.aggregate()

INFO: 2024-12-19 13:59:09,718 - ProMCDA - Number of alternatives: 3
INFO: 2024-12-19 13:59:09,721 - ProMCDA - Number of indicators: 2
INFO: 2024-12-19 13:59:09,722 - ProMCDA - Polarities: ('+', '-')
INFO: 2024-12-19 13:59:09,723 - ProMCDA - Weights: [0.5, 0.5]
INFO: 2024-12-19 13:59:09,724 - ProMCDA - Normalized weights: [0.5, 0.5]


,minmax_weighted_sum,minmax_geometric,minmax_harmonic
0,0.55,0.593296,0.591597
1,0.0,0.100000,0.1
2,1.0,1.000000,1.0


### Test aggregate with sensitivity on normalization

In [10]:
promcda.normalize()
promcda.aggregate(aggregation_method=AggregationFunctions.WEIGHTED_SUM)

INFO: 2024-12-19 13:59:21,638 - ProMCDA - Number of alternatives: 3
INFO: 2024-12-19 13:59:21,639 - ProMCDA - Number of indicators: 2
INFO: 2024-12-19 13:59:21,640 - ProMCDA - Polarities: ('+', '-')
INFO: 2024-12-19 13:59:21,641 - ProMCDA - Weights: [0.5, 0.5]
INFO: 2024-12-19 13:59:21,641 - ProMCDA - Normalized weights: [0.5, 0.5]


,minmax_weighted_sum,target_weighted_sum,standardized_weighted_sum,rank_weighted_sum
0,0.55,0.5625,0.066227,2.0
1,0.0,0.125,-1.029813,1.0
2,1.0,0.916667,0.963586,3.0


### Test aggregate with robustness on weights, fix methods

In [11]:
promcda = ProMCDA(
    input_matrix=setup_no_robustness_indicators['input_matrix'],
    polarity=setup_no_robustness_indicators['polarity'],
    robustness_weights=True,
    robustness_indicators=setup_no_robustness_indicators['robustness_indicators'],
    marginal_distributions=setup_no_robustness_indicators['marginal_distributions'],
    num_runs=setup_no_robustness_indicators['num_runs'],
    num_cores=setup_no_robustness_indicators['num_cores'],
    #output_path=setup_parameters['output_path']
)

INFO: 2024-12-19 14:00:00,122 - ProMCDA - Alternatives are ['A', 'B', 'C']


In [13]:
promcda.normalize(NormalizationFunctions.MINMAX)
promcda.aggregate(aggregation_method=AggregationFunctions.WEIGHTED_SUM)

INFO: 2024-12-19 14:00:11,805 - ProMCDA - Number of alternatives: 3
INFO: 2024-12-19 14:00:11,807 - ProMCDA - Number of indicators: 2
INFO: 2024-12-19 14:00:11,808 - ProMCDA - Polarities: ('+', '-')
INFO: 2024-12-19 14:00:11,812 - ProMCDA - Start ProMCDA with uncertainty on the weights
INFO: 2024-12-19 14:00:11,814 - ProMCDA - All weights are randomly sampled from a uniform distribution.


'Aggregation considered uncertainty on all weights, results are not explicitly shown.'

In [27]:
means, normalized_means, std= promcda.get_aggregated_values_with_robustness_weights()
print("Mean scores are: ")
print(means)

Mean scores are: 
  ws-minmax_01
0     0.540678
1          0.0
2          1.0


### Test aggregate with robustness on weights, sensitivity

In [29]:
promcda = ProMCDA(
    input_matrix=setup_no_robustness_indicators['input_matrix'],
    polarity=setup_no_robustness_indicators['polarity'],
    robustness_weights=True,
    robustness_indicators=setup_no_robustness_indicators['robustness_indicators'],
    marginal_distributions=setup_no_robustness_indicators['marginal_distributions'],
    num_runs=setup_no_robustness_indicators['num_runs'],
    num_cores=setup_no_robustness_indicators['num_cores'],
    #output_path=setup_parameters['output_path']
)

INFO: 2024-12-19 14:15:02,783 - ProMCDA - Alternatives are ['A', 'B', 'C']


In [30]:
promcda.normalize()
promcda.aggregate()

INFO: 2024-12-19 14:15:19,334 - ProMCDA - Number of alternatives: 3
INFO: 2024-12-19 14:15:19,334 - ProMCDA - Number of indicators: 2
INFO: 2024-12-19 14:15:19,334 - ProMCDA - Polarities: ('+', '-')
INFO: 2024-12-19 14:15:19,337 - ProMCDA - Start ProMCDA with uncertainty on the weights
INFO: 2024-12-19 14:15:19,338 - ProMCDA - All weights are randomly sampled from a uniform distribution.


ValueError: Unable to coerce to Series, length must be 8: given 2

## TEST ROBUSTNESS INDICATORS

In [31]:
promcda = ProMCDA(
    input_matrix=setup_robustness_indicators['input_matrix'],
    polarity=setup_robustness_indicators['polarity'],
    robustness_weights=setup_robustness_indicators['robustness_weights'],
    robustness_indicators=setup_robustness_indicators['robustness_indicators'],
    marginal_distributions=setup_robustness_indicators['marginal_distributions'],
    num_runs=setup_robustness_indicators['num_runs'],
    num_cores=setup_robustness_indicators['num_cores'],
    #output_path=setup_parameters['output_path']
)

INFO: 2024-12-19 15:55:33,867 - ProMCDA - Alternatives are ['A', 'B', 'C']


### Test normalize with sensitivity

In [38]:
promcda.normalize()
promcda.get_normalized_values_with_robustness() # If robustness_indicators

[{'standardized_any':   0_standardized_any 1_standardized_any
  0           0.107857           0.228287
  1          -1.049557          -1.094406
  2           0.941699           0.866119,
  'standardized_without_zero':   0_standardized_without_zero 1_standardized_without_zero
  0                    1.257414                    1.422692
  1                         0.1                         0.1
  2                    2.091256                    2.060524,
  'minmax_01':   0_minmax_01 1_minmax_01
  0    0.581248    0.674663
  1         0.0         0.0
  2         1.0         1.0,
  'minmax_without_zero':   0_minmax_without_zero 1_minmax_without_zero
  0              0.623123              0.707196
  1                   0.1                   0.1
  2                   1.0                   1.0,
  'target_01':   0_target_01 1_target_01
  0    0.679686    0.586277
  1    0.235073         0.0
  2         1.0    0.868992,
  'target_without_zero':   0_target_without_zero 1_target_without_zero
  

### Test normalize with specific method

In [33]:
promcda.normalize(NormalizationFunctions.MINMAX)
promcda.get_normalized_values_with_robustness() # If robustness_indicators

[{'minmax_01':   0_minmax_01 1_minmax_01
  0    0.581248    0.674663
  1         0.0         0.0
  2         1.0         1.0,
  'minmax_without_zero':   0_minmax_without_zero 1_minmax_without_zero
  0              0.623123              0.707196
  1                   0.1                   0.1
  2                   1.0                   1.0},
 {'minmax_01':   0_minmax_01 1_minmax_01
  0    0.414038    0.622708
  1         0.0         0.0
  2         1.0         1.0,
  'minmax_without_zero':   0_minmax_without_zero 1_minmax_without_zero
  0              0.472634              0.660437
  1                   0.1                   0.1
  2                   1.0                   1.0},
 {'minmax_01':   0_minmax_01 1_minmax_01
  0    0.477656    0.687736
  1         0.0         0.0
  2         1.0         1.0,
  'minmax_without_zero':   0_minmax_without_zero 1_minmax_without_zero
  0               0.52989              0.718962
  1                   0.1                   0.1
  2                  

### Test aggregate with robustness - need setUpRobustnessIndicators

In [34]:
promcda.normalize(normalization_method=NormalizationFunctions.MINMAX)
promcda.aggregate(aggregation_method=AggregationFunctions.WEIGHTED_SUM, weights=None)

INFO: 2024-12-19 15:55:49,075 - ProMCDA - Number of alternatives: 3
INFO: 2024-12-19 15:55:49,076 - ProMCDA - Number of indicators: 2
INFO: 2024-12-19 15:55:49,076 - ProMCDA - Polarities: ('+', '-')
INFO: 2024-12-19 15:55:49,076 - ProMCDA - Weights: [0.5, 0.5]
INFO: 2024-12-19 15:55:49,076 - ProMCDA - Normalized weights: [0.5, 0.5]
INFO: 2024-12-19 15:55:49,077 - ProMCDA - Start ProMCDA with uncertainty on the indicators
INFO: 2024-12-19 15:55:49,077 - ProMCDA - The number of Monte-Carlo runs is only 5
INFO: 2024-12-19 15:55:49,077 - ProMCDA - A meaningful number of Monte-Carlo runs is equal or larger than 1000


'Aggregation considered uncertainty on indicators, results are not explicitly shown.'

In [36]:
#promcda.get_aggregated_values_with_robustness()